In [13]:
# pip install python-amazon-sp-api

from datetime import datetime, timedelta

import dateutil
from sp_api.api import Orders, Inventories, Reports, CatalogItems
from sp_api.base import SellingApiException, Marketplaces, ReportType, ReportStatus
from sp_api.base.InventoryEnums import InventoryGranularity

In [14]:
from sp_api.base import  SellingApiRequestThrottledException, SellingApiForbiddenException

spapi_brands_config = {
    "PROBIKETOOLS_US": {
        "refresh_token":"Atzr|IwEBINLvV3npPa5YE7rcJJvK2a9Hkx_kTc_b-0kfStArtqqfb0gq-B6_uN5kTJVd3hQ9opyt0GS5QVCpIpfznVSbuB2Vxdoyz3r_3qSf21E6TVDfu_YXSEFlW_94QU-hpsWeyUv-3VSSmbWDqhOmu_NGqe49Rcd6KvQkMeDqtUZlCKTFPFa3OR8aLa7ib45dc_wGAfcV0gCdBfVlBdIvmMsLrX9g0-y6hdJUK3OFB9-xGPhh12Pni8UM1Vyf_Zx1gynd_YnZDIdFmmJ96OIHoqC0ng_Ds75AsJSVryFj4yRU9xQTQrqZ_9IWJVnXPp2DqAE3ebE",
        "lwa_app_id":"amzn1.application-oa2-client.41c0fb4fe5204c5b98c8c4acfa684dd0",
        "lwa_client_secret":"acba7d67bfb0ba603474d7f6bec37e476814461d31738dea1180fce5d089e8fc",
    },
    "PROBIKETOOLS_EU": {
        "refresh_token":"Atzr|IwEBIKRp3roafKM3trTsyLA1uMHHPibkIPYMM1hxPz0kmAeDJWFqaniLy9lLWoV9jVLSNMvvgKlI5tqSrCBHEyKa_GO3UyPaXl54YN5WN4PCUDG6Bq7vFiXmduSEWmVfBbssoHWZIrR5FpJqhf5hUU12sQLbq7ojBb24-Cyl7VGXmqiVVQ3kxnQy4VMSbGBO22RaJjlCa7VkAYtaRbVEusQPnHgXqYHleBIOd2oU0XAdqSymsFv-_tJaSIKoY9-h3BFdFsVnW5UnOwK7pHlV5y9G18p6WrIgDZbwOxOVPJyz6gdqJOxILP6911pg7wba6VZ2yXY",
        "lwa_app_id":"amzn1.application-oa2-client.41c0fb4fe5204c5b98c8c4acfa684dd0",
        "lwa_client_secret":"acba7d67bfb0ba603474d7f6bec37e476814461d31738dea1180fce5d089e8fc",
    },
    "BLACKRHINO_EU": {
        "refresh_token":"Atzr|IwEBICI6XJhHefvEuk4S94r4fUBZjGDVnyOMME4zemudYLqNd2NsIG8xne8QE4JGRt3dsc39xE08IiUVNCJCFTxjGYsotSmEV0LeH8apyzYXZ7ZzF74xpUGVjy7ZwI-GTvLh4UAu3nLxCV-wcKtUJ4BXZCxOVxTtdSMDcBWGfo9wiGDX9n903hKcIGz2-0WJZ2bCtJ0m63fXitfDqmrp0GG_bs81Mp8OkLrEtkWlpNQla5EkFWwGJVTftSp2QHMKLrh50emkfMSJh0aNfmHnGYc_p3qq6jDmBWoleUtE-5VotqFu2vEvJnQDXbOEAkA_Emxjp5w",
        "lwa_app_id":"amzn1.application-oa2-client.de01e5ee80e843179519168f3a8a20e7",
        "lwa_client_secret":"0f43cea65c41f28da3232944b4aaaf257fc01b77349e9e27a79c6ce6ee9d813a",
    },
    "BLACKRHINO_US": {
        "refresh_token":"Atzr|IwEBIFTAGMfmGRHUvPowi7IcSeounSGlsTw9bRhifN5lx8KYjZ5gXSLm_VeB-RxBvqIeaeufAhl0MCJ8ficUqzMs_JXy3NwLqEoEr_PUlS6HGOfIsTayjyaXiKSG4We4S92-Qvk_OP_hlfVdrRifKxOS4mdt0QdlRr47j_jOUDoa7eTY5Ebi9Mh7oe2Qe_aNvZfcU3gGNvbafl2kfTkOxKkPurXylzwbxNBN8_frISYFG_46QSsfrTU6hU01Od1p-uxYv5OSfqbyxVHYCQzVgntfmV272R2tVHYEf7BzSQSEo19QWnAAFKXtbpxpv6VakRlRntQ",
        "lwa_app_id":"amzn1.application-oa2-client.de01e5ee80e843179519168f3a8a20e7",
        "lwa_client_secret":"0f43cea65c41f28da3232944b4aaaf257fc01b77349e9e27a79c6ce6ee9d813a",
    },
    "BARVIVO_EU": {
        "refresh_token":"Atzr|IwEBIP_2JwnWVVg0gjHZwFuOTL0I-zNSTuRcVF23ETD7SlQme5q4vRsYIR8fP7lxctaUQeiheySALcq4JdqIO7aZw9zKElSVQiNkRRV1fkCJKOAFT9IuFCncqi7uv_VKqYKu5VmebKZ1DwFVWTuC2LkkAHZrxdGQX3teX2VmBhAbUP4OTM1_7dPMPQd4hwF6cm1wPMX3w82PBaMtUmWCRhbLxnkTFnbBj_GDyocEQh4OkRUlxCZ3LRIqyWYzHLSlBsasaQ5Tu-WyOG05-s2wsAv1qJsgTC39cJK6QT4Cbakwa6PPLp52m_dJrtu4pn-aGonFkq6AtkHDuruJkh1S2znZ_CL0",
        "lwa_app_id":"amzn1.application-oa2-client.259b7038a46d4c4eabf6bb6022a0db8b",
        "lwa_client_secret":"4b0aecad106211e74e30c7fc5962d7890c2eb17cc4fa96685a2190fa47c5d55e",
    },
    "BARVIVO_US": {
        "refresh_token":"Atzr|IwEBIC-ib6ePw3AfNsTqWyNXlMWlCHAVQII8N6CvG60YoiP4bmDqHfie0_De_Z_vMmSBgvHyKXxE-PDXZmyurqlmMo_ffyFlWrsjMKPjskRuxRQ9mDyM-nzRaUNTDvyCZwUEdQGjR7q46J7mGj_rF-B8QsieiDJNtPswWm3BvI6OKEOqkd5M0fWRCJlINSXdUTby8zHw-XBYFKp7iSi10TYDh3cwHqemMeBunYWvOO4NPDG0FPH-BjVLBERSfyG79jO9gcXKawP37xKqbx5_2R8TSfFI_tgOY-Ec83ZJy8bA07Yvq1nYGxwKyxuoW2A9XfKV0zs",
        "lwa_app_id":"amzn1.application-oa2-client.259b7038a46d4c4eabf6bb6022a0db8b",
        "lwa_client_secret":"4b0aecad106211e74e30c7fc5962d7890c2eb17cc4fa96685a2190fa47c5d55e",
    },
    "OUTLERY_US": {
        "refresh_token":"Atzr|IwEBIN2QzGgfDDmyywwOY5R7grAfkDByCbgz6rA-hzjK7YwLgWZvRcXVWhw-9PfFM63MoN1U8UBB7d5idhoRvm5AUhqWnMzqEE_8PtDdXR_Fb2zBM7Xk-D2ZoZYdrtHn5-A_CE4mPVqwx9yE3Big-gLRyddIb4I7q7XX5xm4SH-GrsfRNCM5Op125vfso8ug74aPTW-gj3U7gKMkVCmenoNTq72uMSfd9oCktD3ZT8UP8rR0qR8aFRaJPwuWH0DmbmkQFIkd3j60VomBgfAAZhGe8IyfOdzVhRm2ss9tJO_GtHZi9Su8zJTr3WtHxJw12zY_dwA",
        "lwa_app_id":"amzn1.application-oa2-client.f2a1d6787dc7431a90e924f53afa47b0",
        "lwa_client_secret":"46bbf202f44adbecb6489eef7ee2fec755da8dc2741a2636ee35ff98d5933566",
    },
    "OUTLERY_EU": {
        "refresh_token":"Atzr|IwEBIFCvHVHVfIgDWCNEL20JeBU5A_EuovpvGZgm1NRKe2YZRLn1LKutH0XpQ2bbKYIkRhoSK-WTZ2expuyTUh3VLXmFKQr6227Ji5Zdk596RLiPSZZ37-45vkwVn3t1sltpKz-0wxCroJ8b40n3AEuXjY9UMlPr0V_4tOSolFypkSk09hUwV6wi5-ePoYBeOHKx4Iwb4KYboAJTQc2CgQDoJ0TjBny103UlMHS01xZUrOsYLRvE1YiL1SGKK36edA88n6JT04FEzdErYRiCdwo4hh_jT6hdh_vKmD_1YqFXtL6bl5n0UlneCO69ouXz1K5ZluBcYGyv_Srsm7vckKtD3yVS",
        "lwa_app_id":"amzn1.application-oa2-client.446d4d90f1984248827aa64d605de4ff",
        "lwa_client_secret":"8f39729f5beeb5dc21bd0aa1cace62512e4502443daef8fde2497ded8121f103",
    },
    "SILVON_US": {
        "refresh_token":"Atzr|IwEBIEZCuQcuov62w9pRERztQ7dZGmlkdLM9GJ6oOgf6iu3Q8IBTPCoi0yHqgmcG52hag_TD0onaLbT1m-9ehD-6dl1Amk7Xd49DvcA3xeHaEPNNS_uh8FupHROHMQCDlvND1IumFEPLa-ikK-6-7i6YEzT7BuGGVi8cVrTb43Sli7UKcYYDgPf1fSqK3VtXUfdT-WMrIzGkIDQOYNeYCXzimksoKFwqRGvYXe1B7fH-tF-pOCHQXn3fJUIvpLRqeu3q-z8JwzQdiMlPm6IOEHu6_kif9Wd2SMiwMsns9QosRmzGww6jVPBAVrBvSIIYvc58B0c",
        "lwa_app_id":"amzn1.application-oa2-client.828f8cf18b8648338b7c7ab15cdc6a5c",
        "lwa_client_secret":"f475259ec39ac341c01b218a12add7fa1b83d9b9a1dea40ceaace75f75bd835f",
    },
    "TOOLZILLA_EU": {
        "refresh_token":"Atzr|IwEBIDMvuo7GuqFJjo60y8oyiNYlVjouz4TDaxefLTvyB0e0AONVDypJWuzzm-4qnIVvdh4EOsZWordlZqdwFPlx0w2WDiPB9xTX1AGUsaOr1ge0cWp1k2VeVd0vXrZHl1hI9wLaFbHwV4FOhiVceZum9IhYY7yX3puQWfrRjvZfbDPBqM15nYHdAaKUMs3nbCnEPMIphWO0mTu24jmTYpWHIqO4I_cVHB2uH8zaJglosDaW1DJQoIZ6W9kXx4_Pm6bpQhbgP_8SwtiI93CnBt1Us1sjVNpQp6osEM2XIR-BRnrcqj_u2lz2tDJrFjNkxgz1wfE",
        "lwa_app_id":"amzn1.application-oa2-client.c346a8c7cf634e8d8974e78ce922a2dd",
        "lwa_client_secret":"58993f8b8c574884607fdac38a304972b0c8ae959894cdd161fb4d3d582c04fb",
    },
    "TOOLZILLA_US": {
        "refresh_token":"Atzr|IwEBIFALDgGQIO1hlHhs4A4KYnLXjE3_K6Ugi93rCvRdReHVuNMNRVpu7BSvJpacE1TcI1OUPB7N8_cFUXs9anM02jRZPgDTs-mw3WclzLqp8wqsSPxDOckg-ILH4IQ3IB86QQyTW_y6CgibZeVyrBgqTi68G8rMXYSdaXzwZ5DHCnfuVej-oLaJBIQcPvD7fM2sT9Ixx2RbMg8gMMEUyfiyWo9rAyMx4lfwYdzAMPvX_aTMoB5pl0xSODie7QEcCY12I2WsY53co01BkyA8eXVNZqmQOAIaVKsfKczqq3ydlXrS1j7hRnGXIJbeZxkNDurSkLk",
        "lwa_app_id":"amzn1.application-oa2-client.c346a8c7cf634e8d8974e78ce922a2dd",
        "lwa_client_secret":"58993f8b8c574884607fdac38a304972b0c8ae959894cdd161fb4d3d582c04fb",
    },
    "WWO_EU": {
        "refresh_token":"Atzr|IwEBIDQiUvn87V_Vn1spAOVKW6zo0MS3Ho3U-LpkJFFbmRIaQGZsoEoRiwUTpdcUsBBmU6N1dVc3EQPK5hWMrZMx4aPaMO1V-3Hhp9GEmhAPqsaFRzBzu5tstIKeD6coLJMouDyNIohhjjbiQww_SixtpKWE7vX1253_CbEOmJdLoeTEqjdNjbXSrmTfZF4qG4jSiZB2fZ7iKZF0PfaAcAWJl6eoxvgNZ3cr1iWuxifG8ROcRQuidkGN4DQJ_PrP5605L4G-WTp0P4zNuZPS8FQQ-mWghgulguuDto_N8XQJYsUHW4dLHSRYu1n-C1bqqiafwNuYuxvZP-3SAdCfLZDKIqgs",
        "lwa_app_id":"amzn1.application-oa2-client.e42981dd55a6471f92c4cc6d0cca6b3d",
        "lwa_client_secret":"e2e2328cc2fb1ffd0ad43b3d44f910b650939bfe36a5c048c8dc667f2a7df22f",
    },
    "WWO_US": {
        "refresh_token":"Atzr|IwEBINEcr_wtQft508SHrsGwlURxrc0G1j-k9aEz5MJkYgvTMch1uWAqWKJ5qa8yWrHwL12aMe_SpUvM_tGBQgrt5VAMY24lwyprMRDPpPqBzUjk45cBiE2igsXyJ7qqeyBLdmAAjFEzlkBRK2FFYNK9xeLwCbNssKw8hVUN_xIOKAZ-rnN3xeLuvzXaAm8DRy2nVJofdq5_eQ_tp0Ucxd4nt3NfaJkvKUp-6XfkzEbERKENk79OrIfIo_G2l5KAylHgVAz-I1SVc2CD4acK3ZT0tPBR1H_F_BIkzYWwcyCZMSZbAZKs1q1qQeA48ZBc1ngUa8A",
        "lwa_app_id":"amzn1.application-oa2-client.e42981dd55a6471f92c4cc6d0cca6b3d",
        "lwa_client_secret":"e2e2328cc2fb1ffd0ad43b3d44f910b650939bfe36a5c048c8dc667f2a7df22f",
    },
    "SWEETVIBES_EU": {
        "refresh_token":"Atzr|IwEBIGwVBREK_E15t98XMWeGw_vpgfAo046c-I3rldsd9fMuq5qynJgK_EVvy1RJLVrT6mdZRwExwvdpQj_flsbIh36yULzm3wN6uGsb65CwNLHnIt-XthQMvaRSGT9xH4T2WcACR8vNPsbW6KQgEjkJ8Qm8yTlIeqjyNXOiH9Yd8hMM-D9k_Pyp6yGTkV9pXUnCHFT9wikQvzsz9MJ0CLyknM5ZO96MgJY4sl_ui_NWTnwI0WNS-78b62zDLINUO4A9jv2MBeAPYq_KcBWYYoMlSq23oP_GBJbkCOEZs8wsg07dGpEZ3c3OkiPGRcfUKCMc0wU",
        "lwa_app_id":"amzn1.application-oa2-client.da0b9ea228b24d748b9c78701b432319",
        "lwa_client_secret":"d8a1a33f9ca5668e0cd03fdc6c4bc2f30b669b460f2f8a307bdc480fe8d92a33",
    },
    "SWEETVIBES_US": {
        "refresh_token":"Atzr|IwEBIFSwVVURBqxpMmy5pbFGmxGn99DeGE7CiwtzsnGQTGpaYVBVJq17XW-L33nm8H4kLK0UvIjmYkPq5CCs6V4Gr7QfZB8kMUu1uMxYX2p7o7zRcK9EyidiEZTqo96U70Pohtt2rh8wa1jIGEtHFUZxUgtGrqnVii74TEpWIqpriPfIm_ftCLePkunl9b2BCasDbn4iiT4rl5AOd2zLDAD2-Qi9mO5J7ipGI9pMQz4WN1-MQcIaI9mFhcbfX3Wig_h-PsV1xWQDVKlcAVrionPFdzwcaRHufCcZAR6DtIEobCWFaL5NOFm8gDWXcs-b03v8W9M",
        "lwa_app_id":"amzn1.application-oa2-client.da0b9ea228b24d748b9c78701b432319",
        "lwa_client_secret":"d8a1a33f9ca5668e0cd03fdc6c4bc2f30b669b460f2f8a307bdc480fe8d92a33",
    },
    "TRIBEWOD_EU": {
        "refresh_token":"Atzr|IwEBIO9MTo1jE8u-sJdX6UjaOuMHBrTp7qH1lxeSsJ3tFUOLShaGyuhaVYRmC1et-qJBZ7LmM5vnXI2n9-OmUY_3bH8jHpYPqNSkVvGki8njyMM-6HcMO-BW9N6wJYVRFtBVm8P7khEERF1hbjzLFOem5SRJwJMUuSFvkHwD_2PtXRQ9TId2ElZP5atM6ucLvvX_CQQFexH81jte0qQcSZNIR-O0OvG6U2XlAa3m-SwJtU3qflN9VL-outnTkv4bYlQIju4OvCAM64k1FpRfhaerG-mACX0wQOp1_tIWt7pUkNt4P0LdfvrMexfQXT1hMVf9i3c",
        "lwa_app_id":"amzn1.application-oa2-client.cec542a11a6a4794a9c2f6f6fa0d36b5",
        "lwa_client_secret":"1c37222f567fdb847ae8bbc15c5a8aaa7c9ffe304fc64fe7622fba6688a56161",
    },
    "TRIBEWOD_US": {
        "refresh_token":"Atzr|IwEBIA5qB_LkW-p6tTfAGIYtCdWVfyB1bYXFseK8qhyOGaekMLMZdyon7PfREcNsGS-PspABAgY3CYsMuDZyD8EC40_DBMYCaTsivYUWHKlm4iytk0rV_FIdOnUim1PW5jqOlU0TKlpr8MPMJ93nWuOorcgqRxqIESj4rVKLGwuYTDIvDgTw4XfhPz2sx60MCK783AJ5QNXfty3vD3Vl1Z9KFir-I0yTyKpSZLL4AGSesYpWOczoyY4KqRuGvStM792pR3Kw6K3mvDhoUmAlvB1EdpWvZIBGhHULpJNGOIat5u5NejVH8hSOUBYu0E5KjIXttd4",
        "lwa_app_id":"amzn1.application-oa2-client.cec542a11a6a4794a9c2f6f6fa0d36b5",
        "lwa_client_secret":"1c37222f567fdb847ae8bbc15c5a8aaa7c9ffe304fc64fe7622fba6688a56161",
    },
    "KETTLEBELLKINGS_US": {
        "refresh_token":"Atzr|IwEBIG4RJYHIveyB1-5v84c-45cxyoN24sH-vFa6HLkVl7uekhzF7LpCwdGEnB5639oRGsD4r94y8nuX-NLtSEvXPDyzJpXoFF2R6FMKe68U2ALLcV9q7JIlsnL5f1jsqNZUimfB-TZ1FB9IC--WCQQ8IliXebn53shFNrzC8PDJ5eGSzOGIWvoJnRI26V9EMl9rHVyGrmGvMiR9WjhR09GVLrTVLePYAtmIQ6TZdiI_Bi0mrUUApNokeFWtcRFoVQCh2F6dr09dn3zXZEj_-xu9OoBCKQh2PfrbNCWgdAd_4PqwwuHXfy9uWCmNChOBmfGdqxo",
        "lwa_app_id":"amzn1.application-oa2-client.ddf5ce77a540453286b64064b3c57286",
        "lwa_client_secret":"98a5266b0d17dfdfd31614de8334261530e52d0ba2a566647eb3b093b64dee67",
    },
    "MOUNTAIN_GRILLERS_EU": {
        "refresh_token":"Atzr|IwEBIHpEJ1bVod8JXk5zRW0S2qliGtyCVo3ydIwY2VYi9QNU1Ijo4oD8rj3K320PVoeFrJTKhK44sLq9GRTsITx_t8c5H4nACjNbZZVrUOlSFFxcAC8N4eIHtTlbL2r_zfxP0GYtYthN2MjVzJr4cHOXpIBfL3QzK8vXaRAfUr3x9cY6ItG2WrxciKi95bj1OTGXN-hYUhpr2qEMJpmFSchuHesOgEB0TRiKF_AT4xiR_1u9ogLqHYAip2iE3L-JyZzt9KbCGkJHOLSFgDs7p1hQoH-ZZvKSm-L3Ik2z8EaPY2kz2KObqjUM3e02aWUIq1i7P4I",
        "lwa_app_id":"amzn1.application-oa2-client.9cd208a8e21f44b39107757d88b36aaf",
        "lwa_client_secret":"ca2ed8f1349260924e1d09b3134b895f19e61126455c9110eb51af4fa0194908",
    },
    "MOUNTAIN_GRILLERS_US": {
        "refresh_token":"Atzr|IwEBIBgHsfIAIm9glS4rU1EpraN5N5V9eQ9CDfrIs_G8mTYffIrwNTketw1Trm0Ojoh23E1WbHSs9bDbbHW7muW0SkiVeMOPWd-X364gETfuNf2fA2anxFQhmY-raigbxzZt7tlOrJF7BxcR6HeLJe-tbTs8cUjsiLyfHz2pYUP-Mu5CsH3jlJ17ASDXvlfLY-CVp9K75DNK43bN4ef3Gwh4mVoW2fBq17QLfxHEmJgI4Ch6AfCMJ3EAyT0HrZb8Or3GO2Xn17VeAC2tOftqILphiZ-Z-MjfIuJnAv0Gaxhb_yBYaCp7w8mJEPjxmZJexmR1LmQ",
        "lwa_app_id":"amzn1.application-oa2-client.9cd208a8e21f44b39107757d88b36aaf",
        "lwa_client_secret":"ca2ed8f1349260924e1d09b3134b895f19e61126455c9110eb51af4fa0194908",
    },
}

def get_client_by(brand_name):
    spapi_brand_config = spapi_brands_config[brand_name]
    credentials = dict(
        refresh_token=spapi_brand_config["refresh_token"],
        lwa_app_id=spapi_brand_config["lwa_app_id"],
        lwa_client_secret=spapi_brand_config["lwa_client_secret"],
        aws_secret_key="",
        aws_access_key="",
        role_arn="arn:aws:iam::798465730220:role/SPAPI_Role",
    )

    market_place = Marketplaces.US if "_US" in brand_name else Marketplaces.GB
    return Reports(
            credentials=credentials,
            marketplace=market_place
        )
    

def get_reports(brand_name, report_name, start_date, end_date):
    while True:
        try:
            reports_client = get_client_by(brand_name)
            return reports_client.get_reports(
                reportTypes=[f'{report_name}'],
                processingStatuses=[ReportStatus.DONE],
                createdSince=start_date,
                createdUntil=end_date
            )
        except SellingApiRequestThrottledException:
            print('[ERROR] limit execeded: getting reports')
            time.sleep(60)

def get_next_page(brand_name, nextToken):
    while True:
        try:
            reports_client = get_client_by(brand_name)
            return reports_client.get_reports(
                nextToken=nextToken
            )
        except SellingApiRequestThrottledException:
            print('[ERROR] limit execeded: getting next page reports')
            time.sleep(60)

In [15]:
import boto3

boto3.setup_default_session(profile_name='data-maintainer-prod')
dynamodb = boto3.resource('dynamodb', region_name='eu-west-1')
table = dynamodb.Table('spapi_reports_config')

def __save_report_created(brand_name: str, report_name: str, start_date: str,
                         end_date: str, report_id: str, report_document_id: str = None):
    table.update_item(
            Key={
                'integration_name': brand_name,
                'report_type': report_name
            },
            UpdateExpression=f"SET reports_created= list_append(reports_created, :map)",
            ExpressionAttributeValues={
                ':map': [
                    {
                        "report_id": report_id,
                        "report_start_date": start_date,
                        "report_end_date": end_date,
                        "report_document_id": report_document_id,
                    }
                ]
            },
            ReturnValues="UPDATED_NEW"
        )

In [19]:
reports_name = ["GET_FBA_FULFILLMENT_CURRENT_INVENTORY_DATA"]
# brands = [ "SWEETVIBES_EU", "SWEETVIBES_US", "TRIBEWOD_EU", "TRIBEWOD_US", "OUTLERY_US"]

brands = [ "SWEETVIBES_US",
           "TRIBEWOD_US",
           "KETTLEBELLKINGS_US",
           "MOUNTAIN_GRILLERS_US",
           "PROBIKETOOLS_US",
           "BLACKRHINO_US",
           "BARVIVO_US",
           "OUTLERY_US",
           "SILVON_US",
           "TOOLZILLA_US",
           "WWO_US",
]

# brands = [ "SWEETVIBES_EU", "SWEETVIBES_US", "TRIBEWOD_EU", "TRIBEWOD_US", "KETTLEBELLKINGS_US", 
# "MOUNTAIN_GRILLERS_EU", "MOUNTAIN_GRILLERS_US","PROBIKETOOLS_EU", "PROBIKETOOLS_US", "BLACKRHINO_EU", "BLACKRHINO_US",
# "BARVIVO_EU", "BARVIVO_US", "OUTLERY_US", "SILVON_US", "TOOLZILLA_EU", "TOOLZILLA_US",
# "WWO_EU", "WWO_US",
#]

#brands = ["PROBIKETOOLS_EU", "PROBIKETOOLS_US", "BLACKRHINO_EU", "BLACKRHINO_US", "BARVIVO_EU", "BARVIVO_US", "OUTLERY_US", "SILVON_US", "TOOLZILLA_EU", "TOOLZILLA_US",
#"WWO_EU", "WWO_US",]

start_report_date = '2022-03-24T00:00:00'
end_report_date = '2022-03-29T00:00:00'

reports = []
for report_name in reports_name:
    for brand in brands:
        report_reponse = get_reports(brand, report_name, start_report_date, end_report_date)
        reports.extend(report_reponse.payload)
        next_token = report_reponse.next_token
        print(len(reports))
        while next_token is not None:
            report_reponse = get_next_page(brand, next_token)
            next_token = report_reponse.next_token
            reports.extend(report_reponse.payload)
            print(len(reports))


        for report in reports:
            print("<><><><><><><><><><><><><><>")
            print(brand)
            print(report_name)
            print(report['dataStartTime'])
            print(report['dataEndTime'])
            print(report['reportId'])
            print(report['reportDocumentId'])
            __save_report_created(brand_name=brand, report_name=report_name, start_date=report['dataStartTime'],
                end_date=report['dataEndTime'], report_id=report['reportId'], report_document_id=report['reportDocumentId'])
            print("<><><><><><><><><><><><><><>") 
        
        reports = []



{'reportTypes': 'GET_FBA_FULFILLMENT_CURRENT_INVENTORY_DATA', 'processingStatuses': 'DONE', 'createdSince': '2022-03-24T00:00:00', 'createdUntil': '2022-03-29T00:00:00', 'path': '/reports/2021-06-30/reports', 'method': 'GET'} None
1
{'nextToken': 'Y6zQebHQmXKudbTFBpV5dpGj5b3tEtak6JRozgJWUDftlwTF2kxe0QqdfR02pdod9o6WXnYNMG1bmOr7aLMxRFf+EmfjpkNoshS8ewEmYLRHk6rHGJE0GfCJzahu/Myb0qd6taus0wqwaEG5roUyW8CsY5Ke1pWI7QwQe1zCovIGjsr2hMe8Ys9BbHy34Cn/P3c65gKFFhzmnf5Q62R9gnacsL7FkeH8D6Z88LN+irDzhbEperI8no/zqbuFDTb0XraGOKm2jL5YcmIHkt0Fs1OQ0RvyTKJGIBIt8xUrqdwdDVlf76a6K11vIAfskSVB1moZuXyhcD1uINpgkhSWv0Dm3ofRgr0SzK5py/naSx7GYNU9DZsFWQ0gPMZrBfEJNY01Q6dpr7rdPEqmnsPjfl2I/8LZMAz5i5liGXvtM5na80vUsWQOp+Z/7fWvKs8ok3CbpsgU/cmvulGCy+otTbkf1v9yze5Y71hGqylzAH3Hhfjyp+Sk/mC4grUFdnAb8CTzx+ym5MY5yezlMhWKChPGMF++GhAfx1T57KrK7UUDspleHn3xDzmkQsP8aN4un0MssKI+ueYt2SfWm2YqXJ/tzOiREzQeq7keoPhYuC1lrIQKtl9waq+6K7r8AOjFd3y8JEUWpX3Usixeh26SyZS+IpRjymDe', 'path': '/reports/2021-06-30/reports', 'method': 'GET'} None


SellingApiBadRequestException: [{'code': 'InvalidInput', 'message': 'NextToken cannot be specified with other input parameters', 'details': 'marketplaceIds;'}]